In [2]:

import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [3]:
from pyspark import SparkContext, SparkConf

conf = SparkConf()
conf.set("spark.app.name", "galina.malyutina Spark RDD Pagerank") 

sc = SparkContext(conf=conf)

In [4]:
import numpy as np
vocabulary = ("Apache", "Spark", "Hadoop")
numbers = np.random.randint(10, size=10000)
words = np.random.choice(vocabulary, size=10000)
collection = zip(numbers, words)

In [5]:
#  Загрузить внешний датасет (датасет загружается из HDFS)
!hdfs dfs -ls /labs/lab06data/ml-100k/u.data
udata= sc.textFile('/labs/lab06data/ml-100k/u.data')

-rw-r--r--   2 hdfs hdfs    1979173 2020-09-30 12:16 /labs/lab06data/ml-100k/u.data


In [78]:
# Оценки лежат в 3 столбце
ratings = udata.map(lambda x : x.split()[2])

In [24]:
# countByValue Return the count of each unique value in this RDD as a dictionary of (value, count) pairs
res = ratings.countByValue()

In [25]:
# sorting
import collections
my_sortedres = collections.OrderedDict(sorted(res.items()))

In [28]:
hist_all=[]
for key,value in my_sortedres.items():
    print ("%s %i" %(key,value))

1 6110
2 11370
3 27145
4 34174
5 21201


In [32]:
# запись в дф
hist_all=[]
for x,y in my_sortedres.items():
    hist_all.append(y)

In [98]:
# 195 id

idfilm="195"

In [118]:
myid = udata.map(lambda x: x.split()).filter(lambda x: x[1] == idfilm)

In [125]:
ratings = myid.flatMap(lambda x: x[2].split())

In [126]:
res = ratings.countByValue()

In [127]:
my_sortedres = collections.OrderedDict(sorted(res.items()))

In [128]:
hist_film=[]
for x,y in my_sortedres.items():
    hist_film.append(y)

In [131]:
dict_to_out = {
    'hist_film' : hist_film,
    'hist_all' : hist_all
}

In [132]:
# выводим в файл
import json

with open('lab06.json', 'w') as outfile:
    json.dump(dict_to_out, outfile)